In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import wandb
from pathlib import Path

In [28]:
api = wandb.Api()

# Project is specified by <entity/project-name>
runs = api.runs("mingfong/fullsim")

summary_list, config_list, name_list = [], [], []
for run in runs: 
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append(
        {k: v for k,v in run.config.items()
          if not k.startswith('_')})

    # .name is the human-readable name of the run.
    name_list.append(run.name)

runs_df = pd.DataFrame({
    "summary": summary_list,
    "config": config_list,
    "name": name_list
    })

runs_df.to_csv("project.csv")

In [29]:
df = pd.read_csv("wandb_fullsim_runs.csv")

In [45]:
for row in df.iterrows():
  current_id = row[1]["ID"]
  print(current_id)
  path = Path("/pscratch/sd/m/mingfong/transfer-learning/wandb/")
  current_wandb_run_dir = list(path.glob(f"**/*{current_id}"))[0]
  artifact = api.artifact(f"mingfong/fullsim/{current_id}-checkpoint:latest")   # TODO figure out which checkpoint had best performance to download
  artifact.download(root=f"/pscratch/sd/m/mingfong/transfer-learning/checkpoints/{current_id}/")

532gak2v


wandb:   1 of 1 files downloaded.  


veyvse59


wandb:   1 of 1 files downloaded.  


b483kdg6


...

wandb:   1 of 1 files downloaded.  


ulw1w4nn


wandb:   1 of 1 files downloaded.  


ujrwfssb


wandb:   1 of 1 files downloaded.  
